In [13]:
# https://github.com/llSourcell/pytorch_in_5_minutes/blob/master/demo.py

import torch
from torch.autograd import Variable
import pandas as pd
from random import randint
from ast import literal_eval
import numpy as np
import torch.nn as nn
# dtype = torch.cuda.FloatTensor # Uncomment this to run on GPU

In [121]:
from torch.autograd import Function
from torch.nn.modules.distance import PairwiseDistance

class TripletLoss(Function):
    
    def __init__(self, margin):
        super(TripletLoss, self).__init__()
        self.margin = margin
        self.pdist  = PairwiseDistance(2)
        
    def forward(self, anchor, positive, negative):
        pos_dist   = self.pdist.forward(anchor, positive).pow(2)
        neg_dist   = self.pdist.forward(anchor, negative).pow(2)
        hinge_dist = torch.clamp(self.margin + pos_dist - neg_dist, min = 0.0)
        loss       = torch.mean(hinge_dist)
        return loss

In [250]:
#read in relevant data
trainData = torch.from_numpy(np.loadtxt('data/trainData.txt', dtype=np.float32))
queryData = torch.from_numpy(np.loadtxt('data/queryData.txt', dtype=np.float32))
df =  pd.read_pickle("./data/KNN.pkl")

In [301]:
# BATCH_SIZE is batch size; INPUT_D is input dimension; OUTPUT_D is output dimension; 
BATCH_SIZE, INPUT_D, HIDDEN_D, OUTPUT_D = 1, 192, 128, 128
ALPHA = 0.5
LEARNING_RATE = 1e-2
K = 5

# Create random Tensor for trainable features, and wrap them in Variables.
# requires_grad=True indicates that we want to compute gradients wrt these Variables during the backward pass.
anchors = Variable(torch.randn(OUTPUT_D, INPUT_D).type(torch.FloatTensor), requires_grad=True)
# weights = Variable(torch.randn(HIDDEN_D, OUTPUT_D).type(torch.FloatTensor), requires_grad=True)

# set b0 to be mean value
aggregate = torch.zeros(OUTPUT_D)
for point in queryData[:5]:
    w0 = torch.norm(point.t() - anchors, 2, 1)
    aggregate += w0
    
biases = Variable((aggregate/queryData.shape[0]).reshape(-1, 1), requires_grad=True)

In [302]:
def generateTripplet(index):
    point = queryData[index].reshape(-1, 1)
    pos = trainData[df.iloc[index].KNN[randint(0,K)]].reshape(-1, 1)
    neg = trainData[df.iloc[index].KNN[randint(K, len(df)-1)]].reshape(-1, 1)
    return point, pos, neg

In [339]:
def forward_pass(query):
    return torch.sign(torch.norm(query.t() - anchors, 2, 1).reshape(-1, 1) - biases)

In [264]:
query = torch.ones(3)
query

tensor([1., 1., 1.])

In [265]:
anchors

tensor([[2.4189, 3.1514, 3.0350],
        [2.9972, 2.3845, 2.2062]], requires_grad=True)

In [266]:
anchors - query.t()

tensor([[1.4189, 2.1514, 2.0350],
        [1.9972, 1.3845, 1.2062]], grad_fn=<SubBackward0>)

In [340]:
#training
for epoch in range(1):
    
    #generate batch and compute collective loss for batch
    batch_indicies = np.random.choice(queryData.shape[0], BATCH_SIZE, replace=False)
    loss = 0
    for index in batch_indicies:
        query, pos, neg = generateTripplet(index)
        queryMapped, posMapped, negMapped = [forward_pass(x) for x in [query, pos, neg]]
        triplet_loss = TripletLoss(ALPHA).forward(queryMapped, posMapped, negMapped)
        loss += triplet_loss
    
    print(epoch, loss.data)

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Variables with requires_grad=True.
    # After this we can call var.grad on variables
    loss.backward()

    # Update weights using gradient descent
    biases.data -= LEARNING_RATE * biases.grad.data
    anchors.data -= LEARNING_RATE * anchors.grad.data

    # Manually zero the gradients 
#     biases.grad.data.zero_()
#     anchors.grad.data.zero_()

0 tensor(0.5000)


In [334]:
torch.norm(query.t() - anchors, 2, 1).reshape(-1,1).shape

torch.Size([128, 1])

In [335]:
biases.shape

torch.Size([128, 1])

In [338]:
(torch.norm(query.t() - anchors, 2, 1).reshape(-1,1) - biases).shape

tensor([[21.7116, 20.0895, 19.4231,  ..., 20.2728, 18.4877, 20.6817],
        [21.7130, 20.0909, 19.4245,  ..., 20.2742, 18.4891, 20.6831],
        [21.7125, 20.0903, 19.4240,  ..., 20.2737, 18.4885, 20.6826],
        ...,
        [21.7117, 20.0896, 19.4233,  ..., 20.2730, 18.4878, 20.6819],
        [21.7128, 20.0907, 19.4243,  ..., 20.2740, 18.4889, 20.6829],
        [21.7130, 20.0909, 19.4245,  ..., 20.2742, 18.4891, 20.6831]],
       grad_fn=<SubBackward0>)

In [341]:
queryMapped.shape

torch.Size([128, 1])

In [348]:
loss = torch.norm(query.t() - anchors, 2, 1).reshape(-1, 1).sum()

In [349]:
print(loss)

tensor(2577.8486, grad_fn=<SumBackward0>)
